In [1]:
# import numpy as np
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import torch
# from sklearn.cluster import DBSCAN
# import annoy
# from annoy import AnnoyIndex
# import os
from tqdm import tqdm
# from shutil import copy

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision import models
# from torchvision import transforms
# from PIL import Image
import pandas as pd


In [138]:
item_pair_set = pd.read_csv("csv file/activation 256dim/resnet_m_2/item_pair_set_top35.csv")
item_pair_set = item_pair_set[['head_label','tail_label']]
item_pair_set

,head_label,tail_label
0,AB13623,BA5115
1,AB13623,CA2249
2,AB13623,BA5263
3,AB13623,BA5167
4,AB13623,BA6363
...,...,...
187350,AB14628,BA4227
187351,AB13877,BA4227
187352,AB13809,BA4227
187353,AB7558,BA4227


In [2]:
mixfitdata = pd.read_csv("csv file/mixFitData.csv")[:5063]
mixfitdata = mixfitdata[['Top', 'Bottom']]
mixfitdata

,Top,Bottom
0,AB10078,BA4933
1,AB12878,BA5666
2,AB12961,BA5674
3,AB12670,BA5684
4,AB12295,BA5685
...,...,...
5058,AB17340,BA6747
5059,AB17376,BA6747
5060,AB17349,BA6780
5061,AB17461,BA6818


In [142]:
3469*10

34690

In [144]:
3469*20

69380

In [146]:
3469*30

104070

In [18]:
for k in [10,20,30,40,50]:#[50]:
    print(f'evaluating top{k} recall...')
    item_pair_set = pd.read_csv(f'img and ch label/csv file/512 tanh/m_3/item_pair_set_top{k}.csv')
    item_pair_set = item_pair_set[['head_label','tail_label']]

    df_len = len(item_pair_set)
    if df_len == (5353*10):
        top_end = (3469*10)
    elif df_len == (5353*20):
        top_end = (3469*20)
    elif df_len ==(5353*30) :
        top_end = (3469*30)
    elif df_len == (5353*40):
        top_end = (3469*40)
    elif df_len == (5353*50):
        top_end = (3469*50)

    top_ans_gr = mixfitdata.groupby('Top')
    bottom_ans_gr = mixfitdata.groupby('Bottom')
    top_pred_gr = item_pair_set[0:top_end].groupby('head_label')
    bottom_pred_gr = item_pair_set[top_end:].groupby('tail_label')

    top_ans_keys = list(top_ans_gr.groups.keys())
    bottom_ans_keys = list(bottom_ans_gr.groups.keys())
    top_pred_keys = list(top_pred_gr.groups.keys())
    bottom_pred_keys = list(bottom_pred_gr.groups.keys())
    topk_df = pd.DataFrame(columns=['query','recall','detail'])

    for keys in tqdm(top_pred_keys, total=len(top_pred_keys)):
        pred_gr = top_pred_gr.get_group(keys)
        pred_list = list(pred_gr.tail_label)
        ans_gr = top_ans_gr.get_group(keys)
        ans_list = list(ans_gr.Bottom)
        correct_ct = 0
        for pred in pred_list:
            if pred in ans_list:
                correct_ct += 1
        topk_recall = round(correct_ct/len(ans_list), 2)#取小數到後面第二位 也可用format方法 出來是字串
        topk_detail = f'{len(ans_list)}中{correct_ct}'
        topk_df.loc[len(topk_df)] = [keys, topk_recall, topk_detail]

    for keys in tqdm(bottom_pred_keys, total=len(bottom_pred_keys)):
        pred_gr = bottom_pred_gr.get_group(keys)
        pred_list = list(pred_gr.head_label)
        ans_gr = bottom_ans_gr.get_group(keys)
        ans_list = list(ans_gr.Top)
        correct_ct = 0
        for pred in pred_list:
            if pred in ans_list:
                correct_ct += 1
        topk_recall = round(correct_ct/len(ans_list), 2)
        topk_detail = f'{len(ans_list)}中{correct_ct}'
        topk_df.loc[len(topk_df)] = [keys, topk_recall, topk_detail]
    
    recall_gr = topk_df.groupby('recall')
    recall_prob = list(set(topk_df.recall))
    recall_prob.sort()
    for i in recall_prob:
        print(f'recall 為 {int(i*100)}% : {len(recall_gr.get_group(i))}筆')

evaluating top10 recall...


100%|██████████| 1884/1884 [00:03<00:00, 484.48it/s]


recall 為 0% : 5185筆
recall 為 2% : 1筆
recall 為 4% : 1筆
recall 為 7% : 1筆
recall 為 8% : 1筆
recall 為 10% : 3筆
recall 為 12% : 3筆
recall 為 13% : 1筆
recall 為 14% : 1筆
recall 為 17% : 5筆
recall 為 20% : 3筆
recall 為 25% : 7筆
recall 為 33% : 12筆
recall 為 50% : 46筆
recall 為 100% : 83筆
evaluating top20 recall...


100%|██████████| 1884/1884 [00:03<00:00, 496.28it/s]


recall 為 0% : 5077筆
recall 為 2% : 2筆
recall 為 3% : 2筆
recall 為 4% : 2筆
recall 為 7% : 1筆
recall 為 8% : 2筆
recall 為 10% : 3筆
recall 為 12% : 4筆
recall 為 13% : 1筆
recall 為 14% : 2筆
recall 為 17% : 6筆
recall 為 20% : 6筆
recall 為 25% : 11筆
recall 為 33% : 26筆
recall 為 40% : 1筆
recall 為 50% : 77筆
recall 為 67% : 1筆
recall 為 100% : 129筆
evaluating top30 recall...


...

recall 為 0% : 4979筆
recall 為 2% : 1筆
recall 為 3% : 3筆
recall 為 4% : 3筆
recall 為 5% : 1筆
recall 為 7% : 2筆
recall 為 8% : 2筆
recall 為 10% : 3筆
recall 為 11% : 2筆
recall 為 12% : 3筆
recall 為 13% : 1筆
recall 為 14% : 1筆
recall 為 17% : 8筆
recall 為 20% : 8筆
recall 為 25% : 23筆
recall 為 28% : 1筆
recall 為 30% : 1筆
recall 為 33% : 29筆
recall 為 40% : 1筆
recall 為 50% : 99筆
recall 為 67% : 3筆
recall 為 100% : 179筆
evaluating top40 recall...


100%|██████████| 1884/1884 [00:03<00:00, 494.13it/s]


recall 為 0% : 4893筆
recall 為 1% : 1筆
recall 為 2% : 3筆
recall 為 3% : 3筆
recall 為 4% : 3筆
recall 為 5% : 1筆
recall 為 7% : 3筆
recall 為 8% : 2筆
recall 為 10% : 3筆
recall 為 11% : 3筆
recall 為 12% : 3筆
recall 為 13% : 1筆
recall 為 14% : 2筆
recall 為 17% : 8筆
recall 為 20% : 8筆
recall 為 25% : 23筆
recall 為 28% : 1筆
recall 為 30% : 1筆
recall 為 33% : 40筆
recall 為 40% : 1筆
recall 為 50% : 129筆
recall 為 67% : 4筆
recall 為 100% : 217筆
evaluating top50 recall...


100%|██████████| 1884/1884 [00:04<00:00, 431.03it/s]

recall 為 0% : 4815筆
recall 為 2% : 2筆
recall 為 3% : 4筆
recall 為 4% : 3筆
recall 為 5% : 2筆
recall 為 6% : 2筆
recall 為 7% : 3筆
recall 為 8% : 2筆
recall 為 10% : 4筆
recall 為 11% : 4筆
recall 為 12% : 1筆
recall 為 14% : 2筆
recall 為 17% : 8筆
recall 為 20% : 10筆
recall 為 25% : 24筆
recall 為 30% : 1筆
recall 為 31% : 1筆
recall 為 33% : 45筆
recall 為 40% : 2筆
recall 為 43% : 1筆
recall 為 50% : 155筆
recall 為 67% : 5筆
recall 為 100% : 257筆


In [5]:
topk_df

,query,recall,detail
0,AB0116,0.0,3中0
1,AB0194,0.0,1中0
2,AB0391,0.0,2中0
3,AB0435,0.0,1中0
4,AB0901,0.0,1中0
...,...,...,...
5348,DA9328,0.0,1中0
5349,DA9337,0.0,2中0
5350,DA9343,0.0,1中0
5351,DA9350,0.0,1中0


In [6]:
topk_df.to_csv(f'512dim_m_3.csv',index=0, encoding='utf-8-sig')

In [10]:
recall_gr = topk_df.groupby('recall')
recall_prob = list(set(topk_df.recall))
recall_prob.sort()
for i in recall_prob:
    print(f'recall 為 {int(i*100)}% : {len(recall_gr.get_group(i))}筆')
    

recall 為 0% : 5309筆
recall 為 2% : 1筆
recall 為 4% : 1筆
recall 為 6% : 1筆
recall 為 7% : 2筆
recall 為 14% : 1筆
recall 為 25% : 2筆
recall 為 33% : 4筆
recall 為 50% : 17筆
recall 為 100% : 15筆


In [11]:
# plt.bar(['0%','8%','12%','33%','50%','100%'], [5328,1,1,4,10,9],width=0.5, bottom=None, align='center', color=['lightsteelblue', 'cornflowerblue', 'royalblue', 'midnightblue', 'navy', 'darkblue', 'mediumblue'])
# # plt.xticks(rotation='vertical')
# plt.show()